# Random graphs

## Erdos-Renyi-model

The Erdos-Renyi-model is constructed as follows:

- We are given $n$ nodes and a link probability $p$, that can be constant or dependent on $n$. 
- For each pair of nodes $\{i,j\}$ we add the (unweighted) link $\{i,j\}$ with independent probability $p$. 

**Expected number of (undirected) links**: $\binom{n}{2}p = n(n-1)p/2$.

**Expected average degree**: $E[\overline{w}] = (n-1)p$.

We now generate an Erdos-Renyi graph and investigate its properties, specifically **connectedness** and **degree distribution**.

### Case 1: constant $p$

In [ ]:
import scipy.io as sio
import networkx as nx
import numpy as np
import scipy as sp
import collections 
import matplotlib.pyplot as plt

# Construction of the Erdos-Renyi model

p = 0.1
n = 20
W = np.zeros((n,n))

# fill the bottom-left block of W
for i in range(n):
    for j in range(i):
        W[i,j] = np.random.choice([0,1], p=[1-p,p])

# fill the top-right block
W = W + W.T
    
G = nx.from_numpy_array(W)

nx.draw(G)

With these parameters, the graph is disconnected with high probability, bacause the expected average degree is small. Let us increase $n$ while keeping $p$ constant.

In [ ]:
p = 0.1
n = 200
W = np.zeros((n,n))
W = W + W.T

for i in range(n):
    for j in range(i):
        W[i,j] = np.random.choice([0,1], p= [1-p,p])
    
G = nx.from_numpy_array(W, create_using=nx.Graph)

nx.draw(G)

print("Expected number of links:", p*n*(n-1)/2)
print("Number of links:", G.number_of_edges())

In [ ]:
print("The graph is connected:", nx.is_connected(G))

Now the plot of the graph is confusing, but the graph is connected.

This shows that $p$ is not the best parameter to capture the connectivity properties of an Erdos-Renyi graph, unless we specify $n$. In the next part, we shall consider cases in which $p$ is a function of $n$.

The problem of keeping $p$ fixed is that the expected average degree grows linearly with $n$, which is quite unrealistic. In fact, typically real networks are sparse.

We study two regimes of interest: 
- $p \propto log(n)/n$ (average degree scaling with $log(n)$), and 
- $p \propto 1/n$ (constant average degree).

### Case 2: $p = a \frac{log(n)}{n}$

Let us generate a large network and count the number of isolated nodes as a first connectivity measure. As we have seen before, if $p$ is fixed and $n$ increases the connectivity of the graph increases. Let us see if this changes when $p$ scales with $n$.

In [ ]:
a = 2
n = 100
p = a*np.log(n)/n

W = np.zeros((n,n))

for i in range(n):
    for j in range(i):
        W[i,j] = np.random.choice([0,1], p= [1-p,p])
        
W = W + W.T
degree = W @ np.ones(n)

nodes_zero_degree = len(degree[degree == 0.])

print("Number of isolated nodes:", nodes_zero_degree)

Let us increase $n$

In [ ]:
a = 2
n_vec = np.arange(100,1100,200)

for n in n_vec:
    
    p = a*np.log(n)/n
    W = np.zeros((n,n))

    for i in range(n):
        for j in range(i):
            W[i,j] = np.random.choice([0,1], p= [1-p,p])

    W = W + W.T
    degree = W @ np.ones(n)
    
    nodes_zero_degree = len(degree[degree == 0.])
    
    print("n:", n)
    print("Number of isolated nodes:", nodes_zero_degree, "\n")

Let us now try different values of $a$ while keeping $n$ fixed.

In [ ]:
a_vec = np.arange(0.55,1.45,0.1)

nodes_zero_degree = []

for a in a_vec:
    n = 600
    p = a*np.log(n)/n

    W = np.zeros((n,n))

    for i in range(n):
        for j in range(i):
            W[i,j] = np.random.choice([0,1], p=[1-p,p])

    W = W + W.T
    degree = W @ np.ones(n)

    nodes_zero_degree.append(len(degree[degree == 0.]))
    
plt.plot(list(a_vec), nodes_zero_degree)

As expected, as $a$ increases the connectivity of the graph increases and the number of isolated nodes decreases.

In fact, one can prove that a phase transition occurs as $a$ varies.

The probability that a node is isolated is $(1-p)^{n-1}$. Thus, the number of isolated nodes in expectation is

$$
E[N_0] = \sum_{i} (1-p)^{n-1} = n(1-p)^{n-1}.
$$

If $p$ is a constant, as $n \to +\infty$ we get $E[N_0] \to 0$.

Instead one can show that if $p = a \frac{log(n)}{n}$, then $E[N_0] \to n^{1-a}$ as $n \to +\infty$, which means that:

- if $a > 1$, $E[N_0] = 0$;
- if $a < 1$, $E[N_0] = +\infty$.

The number of isolated nodes is a measure of connectivity of the graph. Another interesting question is whether the graph is connected or not.

In [ ]:
a_vec = np.arange(0.45,1.55,0.1)

is_connected = []

for a in a_vec:
    # generate the random graph
    n = 600
    p = a*np.log(n)/n

    W = np.zeros((n,n))

    for i in range(n):
        for j in range(i):
            W[i,j] = np.random.choice([0,1], p= [1-p,p])

    W = W + W.T
    G = nx.from_numpy_array(W, create_using=nx.Graph)
    # check whether is connected or not
    is_connected.append(nx.is_connected(G))
    
plt.plot(list(a_vec), is_connected)

One can prove that, as $n \to +\infty$, with large probability:
- if $a>1$, the graph is connected;
- if $a<1$, the graph is not connected.

It can also be proven that the diameter of the graph scales with $log(n)$.

### Case 3: $p = \frac{\lambda}{n}$.

While in case of constant $p$ and $p \propto \frac{log(n)}{n}$ the average degree of the nodes diverges as $n$ grows, here the average degree remains bounded. One can show that the degree distribution follows a Poisson distribution, i.e.,

$$
p_k := \frac{1}{n}|\{i: w_i = k\}| = e^{-\lambda}\frac{\lambda^k}{k!}
$$

In this case the graph is with probability 1 disconnected as $n \to +\infty$. Still, there is another phase transition occurring regarding the size of the largest connected component. Specifically, as $n \to +\infty$:

- if $\lambda<1$, then the size of each connected component i satisfies with probability 1 $C_i \le A log(n)$, i.e., each connected component contains a vanishing fraction of the total number of ndoes;
- if $\lambda>1$, with probability one the largest component has size $C_{max} = n(1-x)$, where $x$ solves $x = e^{\lambda(x-1)}$. Moreover, the size of the second largest component scales with $log(n)$.

The last property is proved by showing that an Erdos-Renyi graph is locally tree-like (i.e., there are no short cycles) and using results from branching processes.

In [ ]:
import collections 

a = 4
n = 1000
p = a/n

W = np.zeros((n,n))

for i in range(n):
    for j in range(i):
        W[i,j] = np.random.choice([0,1], p= [1-p,p])

W = W + W.T
G = nx.from_numpy_array(W, create_using=nx.Graph)

degree = W @ np.ones(n)
degreeCount = collections.Counter(degree)
deg, cnt = zip(*degreeCount.items())

fig = plt.figure(figsize=(8,8))
plt.bar(deg, cnt, width=0.80, color="b")
plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("Degree ER")

Notice that the degree distribution is first increasing and then decreasing, meaning that a large fraction of the nodes has a degree close to the expected average degree, thus the average degree is a good approximation for the degree of a random node of the graph. The variance of the degree distribution is "small".

We shall see in the next lecture that in real networks, as well as other random graphs (e.g., preferential attachment), the variance of the degree distribution is large, i.e., there are a few nodes with very large degree and many nodes with a small degree (compared to average degree).

We finally compute the **clustering coefficient** of the graph, which is the number of triangles in the graph.

In terms of social networks, clustering coefficient in some sense describes "how many friends of a given node are friends to each other".

We expect that clustering coefficient in ER graph is small. Indeed, because of the random structure of the connections there is no reason to expect triangles in the graph.

In [ ]:
clustering_ER = nx.average_clustering(G)

print("Clustering coefficient in ER graph:", clustering_ER)

## Small world networks

Small world networks are characterized by two features:

- small diameter (sublinear in $n$);
- large clustering coefficient (i.e., large number of triangles in the graph).

For the range of parameters $(n,p)$ that make the graph connected, ER graphs satisfy the first condition (the diameter scales with $log(n)$), but not the second one.

Let us see how to generate a small world network. An example is as follows. We start with the following augmented ring.

In [ ]:
n_nodes = 12

G = nx.cycle_graph(n_nodes)

for n in range(n_nodes):
    G.add_edge(n,((n+2) % n_nodes))
    G.add_edge(n,((n-2) % n_nodes))

pos = nx.circular_layout(G)

nx.draw(G,pos,with_labels=True)

Let us generate larger graphs like this, and investigate clustering coefficient and diameter.

In [ ]:
n_nodes_vec = [40,80,160,320,640]

for n_nodes in n_nodes_vec:
    G = nx.cycle_graph(n_nodes)

    for n in range(n_nodes):
        G.add_edge(n,((n+2) % n_nodes))
        G.add_edge(n,((n-2) % n_nodes))

    print("n:", n_nodes)
    print("diameter", nx.diameter(G))
    print("clustering coefficient", nx.average_clustering(G), "\n")

Two observations:
- the clustering coefficient is large and does not scale with $n$;
- the diameter scales linearly with $n$.

To reduce the diameter and obtain a small world network, it is sufficient to add some random long-distance connections.

In [ ]:
p = 0.002
n_nodes = 640

G = nx.cycle_graph(n_nodes)

for n in range(n_nodes):
    G.add_edge(n,((n+2) % n_nodes))
    G.add_edge(n,((n-2) % n_nodes))

for i in range(n_nodes):
    for j in range(n_nodes):
        if np.random.rand() < p:
            G.add_edge(i,j)
            
print("Diameter:", nx.diameter(G))
print("Clustering coefficient", nx.average_clustering(G), "\n")

The new graph has a small diameter. The clustering coefficient is reduced, but still much larger than in Erdos-Renyi graph.

Still, the degree distribution is a shifted Poisson distribution, which differs from the most of the real networks.

### Recap
In the first part of the lecture we have introduced and analysed the properties of two families of random graphs:
- (undirected) **Erdos-Renyi** graph, which is a graph whose links are uniformly random;
- **small world** graphs, in which there is a large fraction of short range links plus a small fraction of random long range links.

Specifically:
- we have shown that both in the Erdos-Renyi and in the small world graphs the **diameter** scales sublinearly with $n$ (specifically, with $log(n)$);
- we have seen that the **degree distribution** of the graphs (for ER this holds $p \propto 1/n$ if is Poisson-like), i.e., it has small variance.
- we have compared the **clustering coefficient** of the two models, i.e., the number of triangles in the graph, showing that in Erdos-Renyi graph the clustering coefficient is much smaller than in the small world graphs.

In the next part we consider a real citation network, and compare how well the random graphs approximate the citation network.
By doing so, we introduce the **preferential attachment model** and the **configuration model** and complete our analysis on random graphs.

## Preferential attachment model

This is a model of growing graph. The idea is the following (we consider the undirected case):

- we start at time $t=0$ with an initial undirected graph $\mathcal{G}_0$;
- at each time $t \in \{1,2,...\}$, we create a new graph $\mathcal{G}_t$ obtained by adding a new node to $\mathcal{G}_{t-1}$, and connecting such a node to $c$ nodes randomly selected. 

If the nodes are selected with uniform probability, we call the model **uniform attachment model**. The more interesting case is when the nodes are selected with a probability which depends on the degree of the nodes, which is called **preferential attachment model**. Several models of preferential attachment may be defined.

In the model introduced by **Price**, the probability for a new node introduced at time $t$ of attaching to a node $i$ is proportional to $a + w_i(t-1)$, where $a > -1$, and $w_i(t-1)$ indicates the degree of node $i$ at time $t-1$.

In the particular case $a=0$ the model is known as **Barabasi-Albert** model.

### Degree distribution of a preferential attachment model
It can be proven that, as $n \to +\infty$, the degree distribution of the Barabasi-Albert model ($a=0$), if each new node attachs to $c$ nodes, follows a power-law distribution, specifically,

$$
p_k =
\begin{cases}
0 \quad &\text{if} \ k<c\\
\frac{2}{2+c} &\text{if} \ k=c\\
\frac{2c(c+1)}{k(k+1)(k+2)} &\text{if} \ k>c.
\end{cases}
$$

Asymptotically for large $k$, this means that $p_k \propto k^{-3}$, which is a power law with exponential $\gamma=3$.

For the more general Price model, it can be proven that for large $k$, $p_k \propto k^{-(3+a)}$.

**Generalizations of preferential attachment model**: 
- this model can be generalized to the case of directed networks, by assuming that the probability of connecting to a node $i$ depends on its indegree $w_i^-$;
- the model can be also generalized by assuming that the number of outlinks of each node is a random variable. This allows to obtain different out-degree distribution (otherwise in directed model all the nodes would have the same out-degree).

We will study other properties of preferential attachment model (e.g., the diameter) in the next part.

# Random graphs as a model for a citation network
The real world network in this problem is a citation network with 10000 papers. It can be loaded with the following code.

In [ ]:
citation = sio.loadmat('citation1.mat', variable_names=['citation'])
citation = citation['citation']

This is a directed network where each node represents a paper and a link from node i to j means that paper i cited paper j. The variable `citation` is an array with 3 columns. Each row contains the tail and head node of a link and the weight associated to such link in the adjacency matrix of the graph (in this case, the weight is either 1 or 0). The graph is constructed from the `citation` variable as follows:

In [ ]:
G = nx.DiGraph()

for i,j,w in citation:
    if w != 0:
        G.add_edge(i,j)

n_nodes_cit = nx.number_of_nodes(G)
n_links_cit = nx.number_of_edges(G)

In the following, we analyze how to approximate the citation graph with different random graph models. We then compare the obtained approximations to see which one is the best for this example.

## Erdos-Renyi-model
To approximate the citation network we construct an Erdos-Renyi-model with as many nodes as the real network, that is $n$. Remember that the expected number of links in Erdos-Renyi undirected graphs is $\binom{n}{2}p$ where $p$ is the probability of there being a link between any two distinct nodes i and j. We choose $p$ such that the expected number of links is the same as the number of links in the actual citation network. By construction, this random graph will not have any self loop.

**Remark**: we here construct a directed Erdos-Renyi graph, since the citation graph is directed. The generalization is straightforward: instead of adding undirected links $\{i,j\}$ with probability $p$, here we add a directed link $(i,j)$ with probability $p$. Thus, the average in-degree and out-degree are both $(n-1)p$, and the expected number of links is $n(n-1)p=2\binom{n}{2}p$, where $\binom{n}{2}p$ is the expected number of undirected links in the undirected Erdos-Renyi graph with $n$ nodes and link probability $p$.

In [ ]:
# Construction of the Erdos-Renyi model

# choose p so that the expected number of links is the same as n_links_cit
# factor 2 at denominator is because G is directed
p = n_links_cit/(sp.special.binom(n_nodes_cit,2) * 2) 
# add links between couple of different nodes with probability p
WER = np.random.choice([0,1], size=(n_nodes_cit, n_nodes_cit), p= [1-p,p])
# GER has no self loops
for n in range(n_nodes_cit):
    WER[n,n] = 0
    
GER = nx.from_numpy_array(WER, create_using=nx.DiGraph)

## The configuration model 
Here we construct a configuration model to approximate the citation network. To do this, we proceed as follows: given the in and out-degree of the real network, we pick uniformly at random one out-link and one in-link and connect them.

In this model, we are able to reproduce the exact in-degree and out-degree distribution.

Instead, in the ER graph, we imposed that the average degree is the same as in the citation graph, but the resulting degree distribution is approximately a Poisson distribution independently of the original degree distribution.

In [ ]:
# Construction of the Configuration model
WCM = np.zeros((n_nodes_cit, n_nodes_cit))
# G.out_degree() is a list of tuples in the form (node,degree)
# we select degree and append to array of degrees
residual_out_degree = np.array(list(degree for node, degree in G.out_degree()))

# same for the indegree
residual_in_degree = np.array(list(degree for node, degree in G.in_degree()))

print(residual_out_degree)

In [ ]:
# for each node
for i in range(n_nodes_cit):
    # while there are unconnected out "half links" starting from the node
    while residual_out_degree[i] > 0:
        available_targets = np.arange(n_nodes_cit)
        # randomly connect the half link to node j, so that the link is (i,j)
        # the probability of connecting to j is proportional to the residual_indegree of j
        target = np.random.choice(available_targets, p = residual_in_degree/sum(residual_in_degree))
        WCM[i,target] = 1
        # decrease out/in degree of the connected nodes
        residual_in_degree[target] -=1
        residual_out_degree[i] -=1

GCM = nx.from_numpy_array(WCM, create_using=nx.DiGraph)

## Preferential attachment model
The preferential attachment is a model for growing random graphs which was explicitly introduced to model citation networks. We now explore this model to see if it is capable of approximating the real citation network.

We generate a random graph by the preferential attachment model, so that the final number of nodes n is the same as in the real citation network and so that the degree distrubution is similar. To do this, we start with directed graph $G_0$ containing 2  nodes connected to each other.

At timestep $t$ $(t\geq1)$, we add a new node to the graph and connect it to $c$ other nodes already present in $G_{t}$, the resulting graph is called $G_{t+1}$. 

The value of $c$ is chosen randomly with probabilities according to the out-degree distribution of the real citation network, so that we obtain the correct out-degree distribution. 

For each of the $c$ new connections, the destination node is chosen with probability proportional to the in-degree distribution of $G_{t}$ plus a constant $a$ , so that the probability that node $i$ is chosen is given by

$$
p_i(t)=\frac{w_i^-(t) + a}{\sum_j (w_j^-(t)+a)}.
$$

**Remark**: when $a \neq 0$, each node has an intrinsic probability of being selected as a neighbor from new nodes, regardless of its in-degree. Observe that each node enters in the network with in-degree $0$. Thus, in the directed Price model $a > 0$, otherwise none of the nodes added at time $t \ge 1$ cannot be chosen by new nodes and cannot increase their indegree.

The process stops when the total number of nodes $n$ is reached.

In [ ]:
# Preferential attachment
GPA = nx.DiGraph()
# GPA is initialized to G0, containing two nodes connected to each other
GPA.add_edges_from([(0,1),(1,0)])

# compute out degree distribution of real citation graph

# degree sequence contains the out degree of nodes ordered from larger to smaller
degree_sequence = sorted([d for n, d in G.out_degree()], reverse=True) 
# count for each out-degree value, how many nodes have that out-degree
# degreeCount is a list of tuples (degree value, number of nodes)
degreeCount = collections.Counter(degree_sequence)
# zip() returns an iterator of tuples where the first item in each passed tuple 
# is paired together, the second item in each passed tuple are paired together
# and so on.
# In this way we obtain deg, which is the tuple of degree values, and
# cnt which is the tuple of the counts.

deg, cnt = zip(*degreeCount.items())
deg_distr = np.array(cnt)/sum(cnt)

print("Degree values in the network:", deg)
print("Counter for each degree", cnt)
print("Degree frequency", deg_distr)

# set a, the intrinsic probability of a node to be selected as a neighbor 
# from new nodes
a = 2

# add the remaining nodes one at a time
for node in range(2,n_nodes_cit):
    # compute degree of new node according to deg_distr (degree distribution
    # of the real citation graph).
    # the min guarantees that the new node is not assigned a degree
    # larger than the current size of the preferential attachment graph
    degree = min(np.random.choice(deg,p=deg_distr), len(GPA)) 
    # choose `degree` neighbors for node to connect to according to their 
    # in-degree in the current approximation GPA.
    # compute updated in degree sequence
    in_deg_PA = [d for n, d in GPA.in_degree()] 
    # add a so that also node with 0 in-degree have non-zero probability
    # of being chosen
    in_deg_PA = np.array(in_deg_PA)+a 
    # normalize to obtain a probability distribution
    in_deg_PA = in_deg_PA/sum(in_deg_PA)
    # replace=False guarantees no neighbor is chosen twice
    neighbors = np.random.choice(np.arange(len(GPA)), p=in_deg_PA, size=degree, replace=False)
    # add the new node (new node is added even if its out-degree is zero)
    GPA.add_node(node)
    # add the new links 
    for neigh in neighbors:
        GPA.add_edge(node,neigh) 
       

## Comparing models
We use the following measures to investigate how the Erdos-Renyi-model, Configuration Model and Preferntial Attachment model are able to approximate the real citation network.
1. Diameter 
2. Average distance between nodes
3. Degree distribution

**Remark:** since the network is not strongly connected, with the usual computation the Diameter becomes infinity. To get a more meaningful result, one can get this diameter as the maximum diameter of each connected component. A similar argument applies to the average distance between nodes.

In [ ]:
# Diameter
length_dict = dict(nx.all_pairs_shortest_path_length(G))
lengths = [lenght for d in length_dict.values() for lenght in d.values()]
diameter = max(lengths)
print("Diameter:", diameter)

length_dict_ER = dict(nx.all_pairs_shortest_path_length(GER))
lengths_ER = [lenght for d in length_dict_ER.values() for lenght in d.values()]
diameter_ER = max(lengths_ER)
print("Diameter ER:", diameter_ER)

length_dict_CM = dict(nx.all_pairs_shortest_path_length(GCM))
lengths_CM = [lenght for d in length_dict_CM.values() for lenght in d.values()]
diameter_CM = max(lengths_CM)
print("Diameter CM:", diameter_CM)

length_dict_PA = dict(nx.all_pairs_shortest_path_length(GPA))
lengths_PA = [lenght for d in length_dict_PA.values() for lenght in d.values()]
diameter_PA = max(lengths_PA)
print("Diameter PA:", diameter_PA)

In [ ]:
# Average distance

ad = np.average(lengths)
print("Average distance:",ad)
ad_ER = np.average(lengths_ER)
print("Average distance ER:",ad_ER)
ad_CM = np.average(lengths_CM)
print("Average distance CM:",ad_CM)
ad_PA = np.average(lengths_PA)
print("Average distance PA:",ad_PA)

In [ ]:
# out-degree

import matplotlib.pyplot as plt

degree_sequence = sorted([d for n, d in G.out_degree()], reverse=True) 
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

degree_sequence_ER = sorted([d for n, d in GER.out_degree()], reverse=True)  # degree sequence
degreeCount_ER = collections.Counter(degree_sequence_ER)
deg_ER, cnt_ER = zip(*degreeCount_ER.items())

degree_sequence_CM = sorted([d for n, d in GCM.out_degree()], reverse=True)  # degree sequence
degreeCount_CM = collections.Counter(degree_sequence_CM)
deg_CM, cnt_CM = zip(*degreeCount_CM.items())

degree_sequence_PA = sorted([d for n, d in GPA.out_degree()], reverse=True)  # degree sequence
degreeCount_PA = collections.Counter(degree_sequence_PA)
deg_PA, cnt_PA = zip(*degreeCount_PA.items())

plt.figure(figsize=[20, 10])

ax = plt.subplot(2,2,1)
plt.bar(deg, cnt, width=0.80, color="b")
plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("Out Degree")
ax.set_xticks([d + 0.4 for d in deg])
ax.set_xticklabels(deg);

ax = plt.subplot(2,2,2)
plt.bar(deg_ER, cnt_ER, width=0.80, color="b")
plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("Out Degree ER")
ax.set_xticks([d + 0.4 for d in deg_ER])
ax.set_xticklabels(deg_ER);

ax = plt.subplot(2,2,3)
plt.bar(deg_CM, cnt_CM, width=0.80, color="b")
plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("Out Degree CM")
ax.set_xticks([d + 0.4 for d in deg_CM])
ax.set_xticklabels(deg_CM);

ax = plt.subplot(2,2,4)
plt.bar(deg_PA, cnt_PA, width=0.80, color="b")
plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("Out Degree PA")
ax.set_xticks([d + 0.4 for d in deg_PA])
ax.set_xticklabels(deg_PA);

**Remark**: Degrees of the real network and of the CM approximation coincide, since the CM model is such that the resulting graph has a prescribed degree distribution. Also the out-degree of the PA model is quite similar, because the out-degree of every node is sampled according to the real out-degree probability distribution. Instead, for the ER the out-degree distribution is Poisson-like.

In [ ]:
# in-degree

degree_sequence = sorted([d for n, d in G.in_degree()], reverse=True)  # degree sequence
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

degree_sequence_ER = sorted([d for n, d in GER.in_degree()], reverse=True)  # degree sequence
degreeCount_ER = collections.Counter(degree_sequence_ER)
deg_ER, cnt_ER = zip(*degreeCount_ER.items())

degree_sequence_CM = sorted([d for n, d in GCM.in_degree()], reverse=True)  # degree sequence
degreeCount_CM = collections.Counter(degree_sequence_CM)
deg_CM, cnt_CM = zip(*degreeCount_CM.items())

degree_sequence_PA = sorted([d for n, d in GPA.in_degree()], reverse=True)  # degree sequence
degreeCount_PA = collections.Counter(degree_sequence_PA)
deg_PA, cnt_PA = zip(*degreeCount_PA.items())

plt.figure(figsize=[20, 10])

ax = plt.subplot(2,2,1)
plt.bar(deg, cnt, width=0.80, color="b")
plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("In Degree")
ax.set_xticks([d + 0.4 for d in deg])
ax.set_xticklabels(deg);

ax = plt.subplot(2,2,2)
plt.bar(deg_ER, cnt_ER, width=0.80, color="b")
plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("In Degree ER")
ax.set_xticks([d + 0.4 for d in deg_ER])
ax.set_xticklabels(deg_ER);

ax = plt.subplot(2,2,3)
plt.bar(deg_CM, cnt_CM, width=0.80, color="b")
plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("In Degree CM")
ax.set_xticks([d + 0.4 for d in deg_CM])
ax.set_xticklabels(deg_CM);

ax = plt.subplot(2,2,4)
plt.bar(deg_PA, cnt_PA, width=0.80, color="b")
plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("In Degree PA")
ax.set_xticks([d + 0.4 for d in deg_PA])
ax.set_xticklabels(deg_PA);

### Exercise
Try to tune the parameters in the PA model (the intrinsic probability of a node to be selected as a neighbor from new nodes, regardless of its in-degree) to obtain a better approximation of the real in-degree distribution.

In [ ]:
# TO DO

In [ ]:
# clustering
clustering = nx.average_clustering(G)
print("Clustering coefficient of the graph:", clustering)

clustering_ER = nx.average_clustering(GER)
print("Clustering coefficient of the ER:", clustering_ER)

clustering_CM = nx.average_clustering(GCM)
print("Clustering coefficient of the CM:", clustering_CM)

clustering_PA = nx.average_clustering(GPA)
print("Clustering coefficient of the PA:", clustering_PA)

The clustering coefficient of the PA model is larger than ER or CM, but still much smaller than the real network.

## Conclusion
The results give an experimental verification of how the preferential attachment model is best suited to approximate citation networks. Indeed, it reaches the best approximation results with respect to all the analyzed measures (for the degree distributions the previous remarks hold).